In [1]:
!wget https://raw.githubusercontent.com/kasprowski/deep2021/main/lecture5/roman/roman_numerals.py

--2024-01-08 22:19:44--  https://raw.githubusercontent.com/kasprowski/deep2021/main/lecture5/roman/roman_numerals.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2161 (2.1K) [text/plain]
Saving to: ‘roman_numerals.py’

roman_numerals.py   100%[===================>]   2.11K  --.-KB/s    in 0s      

2024-01-08 22:19:44 (28.4 MB/s) - ‘roman_numerals.py’ saved [2161/2161]



In [2]:
!wget https://raw.githubusercontent.com/kasprowski/deep2021/main/lecture5/roman/encoder.py

--2024-01-08 22:19:47--  https://raw.githubusercontent.com/kasprowski/deep2021/main/lecture5/roman/encoder.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1812 (1.8K) [text/plain]
Saving to: ‘encoder.py’

encoder.py          100%[===================>]   1.77K  --.-KB/s    in 0s      

2024-01-08 22:19:47 (23.9 MB/s) - ‘encoder.py’ saved [1812/1812]



In [29]:
import random
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from encoder import CharacterTable
import roman_numerals as cnv
import numpy as np
import random

In [30]:
# Parameters for the model and dataset.
DATASET_SIZE = 200
INPUT = 14
OUTPUT = 4
# object to encode roman numbers to one-hot
romans = 'MDCLXVI '
rtable = CharacterTable(romans)

# object to encode arabic numbers to one-hot
chars = '0123456789 '
dtable = CharacterTable(chars)

In [31]:
seq_samples = []
seq_labels = []
used = []
repetitions = 0
generated = 0
print('Generating data...')
while len(seq_samples) < DATASET_SIZE:
    number = random.randint(1,200)
    # skip if already in the dataset
    if number in used: continue
    used.append(number)
    # roman input
    roman = cnv.convert(number)
    roman += ' ' * (INPUT - len(roman))

    # arabic output
    arabic = str(number)
    arabic += ' ' * (OUTPUT - len(arabic))

    seq_samples.append(roman)
    seq_labels.append(arabic)
print('Total roman numbers:', len(seq_samples))
#print("samples",seq_samples)
#print("labels",seq_labels)

Generating data...
Total roman numbers: 200


In [32]:
# one-hot encoding of all romans and arabic numbers
print('Vectorization...')
samples = np.zeros((DATASET_SIZE, INPUT, len(romans)), dtype=np.bool)
labels = np.zeros((DATASET_SIZE, OUTPUT, len(chars)), dtype=np.bool)
for i, sentence in enumerate(seq_samples):
    samples[i] = rtable.encode(sentence, INPUT)
for i, sentence in enumerate(seq_labels):
    labels[i] = dtable.encode(sentence, OUTPUT)
print("samples",samples.shape)
print("labels",labels.shape)

Vectorization...
samples (200, 14, 8)
labels (200, 4, 11)


<ipython-input-32-7911ce97f998>:3: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  samples = np.zeros((DATASET_SIZE, INPUT, len(romans)), dtype=np.bool)
<ipython-input-32-7911ce97f998>:4: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  labels = np.zeros((DATASET_SIZE, OUTPUT, len(chars)), dtype=np.bool)


In [33]:
import sklearn.model_selection
(trainSamples, testSamples, trainLabels, testLabels) = sklearn.model_selection.train_test_split(samples,labels,random_state=42)
print("Training samples",len(trainSamples))
print("Test samples",len(testSamples))

Training samples 150
Test samples 50


In [34]:
# Build the network
print('Build model...')
model = tf.keras.Sequential()
model.add(layers.LSTM(128, input_shape=(INPUT, len(romans))))
model.add(layers.RepeatVector(OUTPUT))
model.add(layers.LSTM(128, return_sequences=True))
model.add(layers.Dense(len(chars), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Build model...
Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 128)               70144     
                                                                 
 repeat_vector_1 (RepeatVec  (None, 4, 128)            0         
 tor)                                                            
                                                                 
 lstm_3 (LSTM)               (None, 4, 128)            131584    
                                                                 
 dense_1 (Dense)             (None, 4, 11)             1419      
                                                                 
Total params: 203147 (793.54 KB)
Trainable params: 203147 (793.54 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [37]:
def avg(list):
    return sum(list)/len(list)

EPOCHS = 50
# Train the model each generation and show predictions against the validation dataset.
for iteration in range(1, 200):
    print()
    print('-' * 50)
    print('Iteration', iteration*EPOCHS)
    H = model.fit(trainSamples, trainLabels, batch_size=128, epochs=EPOCHS,
                  validation_data=(testSamples, testLabels), verbose=0)
    print("Validation accuracy: {}".format(avg(H.history["val_accuracy"])))
    # Select 10 samples from the validation set at random to visualize errors.
    accuracy = 0
    for i in range(len(testSamples)):

        rowx, rowy = testSamples[np.array([i])], testLabels[np.array([i])]
        preds = np.argmax(model.predict(rowx),axis=1)
        roman = rtable.decode(rowx[0])
        correct = dtable.decode(rowy[0])
        prediction = dtable.decode(preds[0], calc_argmax=False)
        correctTxt = ""
        if correct == prediction:
            correctTxt+='OK!'
            accuracy+=1
        v = random.random()
        if v>0.9:
            print('expression:', roman, end=' ')
            print('correct:', correct, end=' ')
            print('predicted:', prediction, end=' ')
            print(correctTxt)

    print('accuracy=',accuracy,"/",len(testSamples),"=",accuracy/len(testSamples))

Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 31ms/step
expression: II             correct: 2    predicted: 1           
1/1 [==============================] - 0s 25ms/step
expression: XLII           correct: 42   predicted: 2  00    10 
1/1 [==============================] - 0s 50ms/step
expression: X              correct: 10   predicted: 10    00 2  
1/1 [==============================] - 0s 37ms/step
accuracy= 0 / 50 = 0.0

--------------------------------------------------
Iteration 5850
Validation accuracy: 0.8999999761581421
1/1 [==============================] - 0s 29ms/step
expression: XXIII          correct: 23   predicted: 1   0    0  
1/1 [==============================] - 0s 50ms/step
expression: CLIII          correct: 153  predicted: 20 11000111 
1/1 [==============================] - 0s 62ms/step
expression: CLII           correct: 152  predicted: 20 11000111 
1/1 [==============================] - 0s 53ms/step
expression: C